## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,42.96,95,100,17.98,moderate rain
1,1,Rikitea,PF,-23.1203,-134.9692,71.22,64,55,17.25,broken clouds
2,2,Tiznit,MA,29.5833,-9.5000,76.28,54,20,11.36,few clouds
3,3,Butaritari,KI,3.0707,172.7902,81.93,76,11,13.42,few clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,60.31,91,75,12.66,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,71.22,64,55,17.25,broken clouds
2,2,Tiznit,MA,29.5833,-9.5000,76.28,54,20,11.36,few clouds
3,3,Butaritari,KI,3.0707,172.7902,81.93,76,11,13.42,few clouds
6,6,Santa Cruz,BO,-17.8000,-63.1667,84.25,54,3,17.27,clear sky
7,7,Vila Velha,BR,-20.3297,-40.2925,73.35,68,75,14.00,broken clouds
13,13,Marrakesh,MA,31.6315,-8.0083,84.27,48,40,3.44,scattered clouds
15,15,Sola,VU,-13.8833,167.5500,80.29,79,99,22.41,overcast clouds
17,17,Nkhotakota,MW,-12.9274,34.2961,83.84,36,48,7.52,scattered clouds
19,19,Labuhan,ID,-6.8844,112.2051,82.47,70,57,7.45,broken clouds
20,20,Cabo San Lucas,MX,22.8909,-109.9124,82.51,70,84,4.36,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                302
City                   302
Country                301
Lat                    302
Lng                    302
Max Temp               302
Humidity               302
Cloudiness             302
Wind Speed             302
Current Description    302
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,71.22,broken clouds,-23.1203,-134.9692,
2,Tiznit,MA,76.28,few clouds,29.5833,-9.5000,
3,Butaritari,KI,81.93,few clouds,3.0707,172.7902,
6,Santa Cruz,BO,84.25,clear sky,-17.8000,-63.1667,
7,Vila Velha,BR,73.35,broken clouds,-20.3297,-40.2925,
13,Marrakesh,MA,84.27,scattered clouds,31.6315,-8.0083,
15,Sola,VU,80.29,overcast clouds,-13.8833,167.5500,
17,Nkhotakota,MW,83.84,scattered clouds,-12.9274,34.2961,
19,Labuhan,ID,82.47,broken clouds,-6.8844,112.2051,
20,Cabo San Lucas,MX,82.51,broken clouds,22.8909,-109.9124,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   284
Country                284
Max Temp               284
Current Description    284
Lat                    284
Lng                    284
Hotel Name             284
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))